## Order and Online customer behavior data analytics

In [1]:
# All basic imports
import numpy as np
import pandas as pd
import os
import zipfile

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import warnings
warnings.filterwarnings('ignore')

In [2]:
# unzip data.zip
zip_ref = zipfile.ZipFile("data.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
# list of files in the working directory
os.listdir()

['.git',
 '.ipynb_checkpoints',
 'Capture.PNG',
 'data.zip',
 'online.csv',
 'order.csv',
 'README.md',
 'sflydatascience-homework1.ipynb']

In [4]:
onlineData = pd.read_csv("online.csv")
orderData = pd.read_csv("order.csv")

In [5]:
onlineData['date'] = 0
onlineData['time'] = 0
onlineData['date'] = onlineData.dt.apply(lambda x : x[0:10])
onlineData['time'] = onlineData.dt.apply(lambda x : x[11:])

In [6]:
onlineData.shape

(954774, 9)

In [7]:
orderData.shape

(263278, 6)

In [8]:
orderData.head()

,custno,ordno,orderdate,prodcat2,prodcat1,revenue
0,18944,64694,2016-11-27 20:57:20,NaN,1,53.30
1,18944,114405,2017-04-29 20:18:04,NaN,1,0.10
2,18944,28906,2017-04-23 21:31:03,NaN,1,141.66
3,36096,62681,2016-02-25 07:16:33,NaN,1,36.82
4,1,1,2017-06-12 08:27:59,NaN,1,8.35


In [26]:
# one or more products were purchased per order as # of unique orders are less than total # of rows in order table
len(orderData.ordno.unique())

149717

In [40]:
onlineData[(onlineData.custno == 637) & (onlineData.date == "2017-10-04")]

,session,visitor,dt,custno,category,event1,event2,date,time
71874,9,405166,2017-10-04 13:36:14,637,2,NaN,3,2017-10-04,13:36:14
135720,9,405166,2017-10-04 13:07:03,637,2,NaN,7,2017-10-04,13:07:03


In [41]:
orderData[(orderData.custno == 637) & (orderData.date == "2017-10-04")]

,custno,ordno,orderdate,prodcat2,prodcat1,revenue,date,time
153499,637,84765,2017-10-04 02:08:35,25.0,7,68.48,2017-10-04,02:08:35
166738,637,151367,2017-10-04 13:47:35,30.0,3,119.46,2017-10-04,13:47:35


In [22]:
onlineData.head()

,session,visitor,dt,custno,category,event1,event2,date,time
0,419542,140970,2016-09-16 05:03:23,3840,1,NaN,1,2016-09-16,05:03:23
1,3030130,14501,2017-11-13 04:58:12,70400,1,NaN,1,2017-11-13,04:58:12
2,2638740,419353,2017-05-24 16:10:38,21248,1,NaN,1,2017-05-24,16:10:38
3,880408,90673,2017-10-28 13:42:38,39168,1,NaN,1,2017-10-28,13:42:38
4,2612179,191542,2016-05-17 06:30:32,47616,1,NaN,1,2016-05-17,06:30:32


In [10]:
orderData['date'] = 0
orderData['time'] = 0
orderData['date'] = orderData.orderdate.apply(lambda x : x[0:10])
orderData['time'] = orderData.orderdate.apply(lambda x : x[11:])

In [11]:
orderData[(onlineData.date == "2016-09-16") & (onlineData.custno == 3840)]

,custno,ordno,orderdate,prodcat2,prodcat1,revenue,date,time
0,18944,64694,2016-11-27 20:57:20,NaN,1,53.3,2016-11-27,20:57:20


In [12]:
onlineData[(onlineData.date == "2016-09-16") & (onlineData.custno == 3840)]

,session,visitor,dt,custno,category,event1,event2,date,time
0,419542,140970,2016-09-16 05:03:23,3840,1,NaN,1,2016-09-16,05:03:23


In [14]:
orderData.head(7)

,custno,ordno,orderdate,prodcat2,prodcat1,revenue,date,time
0,18944,64694,2016-11-27 20:57:20,NaN,1,53.30,2016-11-27,20:57:20
1,18944,114405,2017-04-29 20:18:04,NaN,1,0.10,2017-04-29,20:18:04
2,18944,28906,2017-04-23 21:31:03,NaN,1,141.66,2017-04-23,21:31:03
3,36096,62681,2016-02-25 07:16:33,NaN,1,36.82,2016-02-25,07:16:33
4,1,1,2017-06-12 08:27:59,NaN,1,8.35,2017-06-12,08:27:59
5,6401,8187,2017-03-26 09:13:06,NaN,1,9.90,2017-03-26,09:13:06
6,25601,41198,2017-09-10 19:54:00,NaN,1,132.46,2017-09-10,19:54:00


In [19]:
orderData.groupby(['prodcat1']).size()

prodcat1
1    48672
2    88684
3    44019
4    46681
5    11180
7    24042
dtype: int64

In [ ]:
onlineData

In [13]:
# I want to check if a customer come online but does not checkout
#orderDatatemp = orderData[(orderData.date == '2016-09-16')]
#def wasOnlinebutDidnotorder():
#        for i in onlineData.custno.unique():
#            try:
#                orderData[(onlineData.custno == i)]
#            except:
#                print("customer " + str(i) + " came online and did not checkout")